In [1]:
import sys
import os
import pickle
import csv

from io import BytesIO
import numpy as np

from rdflib import URIRef
from rdflib.namespace import ClosedNamespace


In [2]:
sys.path.append("/home/alireza/aida/git/AIDA-Interchange-Format/python/aida_interchange")
from Bounding_Box import Bounding_Box
import aifutils


In [3]:
with open('../../temp/all_labels_coco_oi_ws.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [4]:
all_labels[1]

'/m/0449p'

In [5]:
with open('../../temp/all_labels_names.pkl', 'rb') as fin:
    mid2name = pickle.load(fin)

In [6]:
mid2term = {item: item.split('/')[-1] + '_' + mid2name[item].replace(' ', '_').replace('&', 'and') for item in all_labels}

In [7]:
with open('../../results/det_results_combined_oi_coco_ws_7_post.pkl', 'rb') as fin:
    det_results = pickle.load(fin)

In [8]:
VIS_ONTOLOGY = ClosedNamespace(
    uri=URIRef("Columbia/DVMM/Ontologies/OpenImagesSubset01#"),
    terms=[mid2term[item] for item in all_labels]
)


In [9]:
g = aifutils.make_graph()

system_co = aifutils.make_system_with_uri(g, "Columbia/DVMM/Systems/ObjectDetection/FasterRCNN-NASNet/COCO")
system_oi = aifutils.make_system_with_uri(g, "Columbia/DVMM/Systems/ObjectDetection/FasterRCNN-InceptionResNet/OpenImages")
system_ws = aifutils.make_system_with_uri(g, "Columbia/DVMM/Systems/ObjectDetection/MITWeaklySupervised-ResNet/OpenImages")

for imgid, det in det_results.items():
    for ii in range(det['num_detections']):
        label = det['detection_classes'][ii]
        score = det['detection_scores'][ii]
        bbox = det['detection_boxes'][ii]
        model = det['detection_models'][ii]
        
        if score < 0.25:
            continue
        
        if model == 'coco':
            sys = system_co
        elif model == 'oi':
            sys = system_oi
        elif model == 'ws':
            sys = system_ws
        elif model == 'coco/J':
            sys = system_co
        elif model == 'oi/J':
            sys = system_oi
        elif model == 'ws/J':
            sys = system_ws

        labelrdf = VIS_ONTOLOGY.term(mid2term[label])
        
        entity = aifutils.make_entity(g, f"Columbia/DVMM/Entities/ObjectDetection/RUN00001/{imgid}/{ii}", sys)

        type_assertion = aifutils.mark_type(g, f"Columbia/DVMM/TypeAssertion/ObjectDetection/RUN00001/{imgid}/{ii}/1", 
                                            entity, labelrdf, sys, score)

        bb = Bounding_Box((bbox[0], bbox[1]), (bbox[2], bbox[3]))
        _, vidid, keyframeid = imgid.split('/')
        aifutils.mark_keyframe_video_justification(g, [entity, type_assertion], vidid, keyframeid, bb, sys, score)
        



In [10]:
with open('../../results/aida_output_7', 'w') as fout:
    serialization = BytesIO()
    # need .buffer because serialize will write bytes, not str
    g.serialize(destination=serialization, format='turtle')
    fout.write(serialization.getvalue().decode('utf-8'))


In [10]:
with open('../../temp/ontology_export_1.ttl', 'w') as fout:
    fout.write('@prefix : <Columbia/DVMM/Ontologies/OpenImagesSubset01#> .\n')
    fout.write('@prefix owl: <http://www.w3.org/2002/07/owl#> .\n')
    fout.write('@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n')
    fout.write('@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n')
    fout.write('@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n')
    fout.write('\n')
    fout.write('<Columbia/DVMM/Ontologies/OpenImagesSubset01#> rdf:type owl:Ontology .')
    fout.write('\n')
    for label in all_labels:
        fout.write(f':{mid2term[label]} rdf:type owl:Class .\n')